In [1]:
import sys
sys.path.append('/home/gbortolai/Thesis/')

In [1]:
import torch
import numpy as np
import ot

In [27]:
#GPU
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
print(f"Using {device} device")

Using cuda device


In [28]:
def wasserstein_dist(a,b,metric, d, reg):
    
    source = a[torch.nonzero(a, as_tuple = False)[:,0], torch.nonzero(a, as_tuple = False)[:,1]].view(torch.nonzero(a, as_tuple = False).size(0))
    target = b[torch.nonzero(b, as_tuple = False)[:,0], torch.nonzero(b, as_tuple = False)[:,1]].view(torch.nonzero(b, as_tuple = False).size(0))
    
    source = source.to(device)
    target = target.to(device)
    
    source = source/torch.sum(source, dtype = torch.double)
    target = target/torch.sum(target, dtype = torch.double)
    
    M = ot.dist(torch.nonzero(a, as_tuple = False).double(), torch.nonzero(b, as_tuple = False).double(), metric = metric).to(device)
    M = M/d
    
    T = ot.sinkhorn(source, target, M, reg)
    # T = ot.emd(source, target, M)
    W = torch.sum(T*M).to(device)
    
    W = torch.sqrt(W)
    
    return W

In [29]:
data, targets = torch.load('data/MNIST/train_s=3000', map_location = device)
d = 2*28*28

In [42]:
import time
#Wasserstein distance 

metric='sqeuclidean'
reg = 1e-1

# size = data.size(0)
size = data[:10].size(0)

Wasserstein_dist = torch.zeros(size, size, dtype = torch.double).to(device)
st = time.time()
for i in range (size):
    for j in filter(lambda h: h>i, range (size)):
        Wasserstein_dist[i][j] = wasserstein_dist(data[i], data[j], metric, d, reg).to(device)
    print('Row n:',(i/size)*100)
    
et = time.time()
print(et-st)

Row n: 0.0
Row n: 10.0
Row n: 20.0
Row n: 30.0
Row n: 40.0
Row n: 50.0
Row n: 60.0
Row n: 70.0
Row n: 80.0
Row n: 90.0
0.15265130996704102


In [ ]:
torch.save([Wasserstein_dist, targets],'data/MNIST/Wasserstein_dist_train_s='+str(size))

funzione per estrarre valori: scipy.sqrform